# Super Image Resolution

In [1]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla V100-SXM2-16GB, 440.33.01, 16160 MiB


In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5 MB 8.0 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4


In [3]:
!pip install numpy opencv-python lmdb pyyaml pickle5 matplotlib seaborn tqdm

     |████████████████████████████████| 876 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 50.4 MB 379 kB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 84.3 MB/s eta 0:00:01
  Created wheel for lmdb: filename=lmdb-1.0.0-cp38-cp38-linux_x86_64.whl size=281039 sha256=8c795e9f27294ea72d9cfbd6f7c7e4faef2e8d30bd119987c0039c8e177aae89
  Stored in directory: /root/.cache/pip/wheels/ef/74/ab/c88a07ddeda6a031f03fd49fda8478ac33ab944e12e04cb0a4
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=268713 sha256=6cc6321bf7c48b25b12214148050aa0374be97a4027e12b50f7067220face069
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built lmdb pickle5


In [ ]:
apt-get update && apt-get install libgl1-mesa-glx ffmpeg

In [4]:
!git clone https://github.com/idealo/image-super-resolution SIR/src

Cloning into 'SIR/src'...
remote: Enumerating objects: 1157, done.
remote: Total 1157 (delta 0), reused 0 (delta 0), pack-reused 1157
Receiving objects: 100% (1157/1157), 14.94 MiB | 11.26 MiB/s, done.
Resolving deltas: 100% (579/579), done.


In [5]:
%cd SIR/src
!pip install -e ".[gpu]"
%cd ../..

/root/SIR/src
Obtaining file:///root/SIR/src
  Running setup.py develop for ISR
/root


In [1]:
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# SIR

In [1]:
################# Required Configurations ############################

INPUT_FILEPATH = "Olvidados30.mp4" #param{type:"string"}

OUTPUT_FILEPATH = "Olvidados30HD.mp4" #param{type:"string"}

################# Optional configurations ############################

INPUTFRAMES_DIRPATH = 'SIR/frames' #@param{type:"string"}

OUTPUTFRAMES_DIRPATH = 'SIR/framesHD' #@param{type:"string"}

In [2]:
# Importing all necessary libraries 
import cv2 
import os 
import numpy as np
import glob
from tqdm import tqdm
import os
from fastprogress.fastprogress import progress_bar

In [9]:
def runnerISR(path_to_images,path_to_output):
    
    from ISR.models import RDN
    import numpy as np
    from PIL import Image

    files = []

    for r, _, f in os.walk(path_to_images):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","")])
    print("")
    files.sort()
    files.sort(key=lambda x: int(x[1]))
    #-----------------------------------------------
    model = RDN(weights='noise-cancel')
    
    #for file in tqdm(range(len(files))):
    progressbar = progress_bar(range(len(files)))
    for file in progressbar:

        path = files[file][0]
        name = files[file][1]
        
        progressbar.comment = "Processing: "+str(path)

        img = Image.open(path)
        sr_img = model.predict(np.array(img))
        image2x = Image.fromarray(sr_img)
        image2x.save(path_to_output+"/"+name+'.jpg','JPEG')
        

In [10]:
rm -R '{INPUTFRAMES_DIRPATH}' && rm -R '{OUTPUTFRAMES_DIRPATH}' 

In [11]:
mkdir '{INPUTFRAMES_DIRPATH}'&& mkdir '{OUTPUTFRAMES_DIRPATH}'

In [12]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
filename = os.path.basename(INPUT_FILEPATH)
!ffmpeg -r 23.976 -i '{filename}' '{INPUTFRAMES_DIRPATH}/%08d.png'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [13]:
runnerISR(INPUTFRAMES_DIRPATH,OUTPUTFRAMES_DIRPATH)

KeyboardInterrupt: 

In [ ]:
%cd {OUTPUTFRAMES_DIRPATH}
! rm 'temp_outputaudio.aac'
! ffmpeg -i '../../{INPUT_FILEPATH}' -acodec copy 'temp_outputaudio.aac'
! ffmpeg -y -r 23.976 -f image2 -pattern_type glob -i '*.jpg' -vcodec libx264 -shortest '../../{OUTPUT_FILEPATH}'
%cd ../..